In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#clasificadores
from utils.clasificacion import *

#adaptacion
from utils.adaptacion import *

#otros
import os
import numpy as np
import pandas as pd
import timeit
from sklearn.externals import joblib

tipo = pruebas[1]
dataset_name = datasets[1]
dims = dimensions[dataset_name]

Using Theano backend.
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## Pruebas con el dataset Twitter

In [3]:
print tipo
print dataset_name
print dims
print data_path

msda
twitter
2000
data


In [4]:
# cargando dataset Twitter
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [5]:
# se obtienen todos los valores X disponibles para realizar adaptacion
X = dataset_object.get_all_X()
X = np.asarray(X)

print "Todos los datos disponibles obtenidos"

Todos los datos disponibles obtenidos


```python

saved_paths = [ruta_1,...,ruta_n]
```

In [6]:
t_total_i = timeit.default_timer()

msda_paths = os.path.join(models_path, tipo, "%s_paths.pkl" % dataset_name)

if os.path.exists(msda_paths):
    print "Cargando rutas de modelos adaptados."
    saved_paths = joblib.load(msda_paths)
else:
    print "Creando modelos de adaptacion..."

    noises = [0.3, 0.5, 0.8]
    layers_sizes = [1, 3, 5]

    parametros = {
        'noises': noises,
        'layers': layers_sizes
    }


    saved_paths = msda_theano_pseudo_grid_search(X, parametros, models_path, tipo, dataset_name)

    print "Modelos creados.\n"
    
    joblib.dump(saved_paths, msda_paths)
    print "Guardando rutas en %s" % msda_paths
    
t_total_f = timeit.default_timer()

t_total = (t_total_f - t_total_i)/60.

print "Creacion de modelos realizada en %.2fm" % t_total

Creando modelos de adaptacion...
	pr: 0.300 - l: 1
	Entrenando capa  1
	Entrenado en 0.87m

	Guardando modelo en models/msda/twitter_0.pkl
	pr: 0.300 - l: 3
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenado en 2.07m

	Guardando modelo en models/msda/twitter_1.pkl
	pr: 0.300 - l: 5
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenando capa  4
	Entrenando capa  5
	Entrenado en 3.32m

	Guardando modelo en models/msda/twitter_2.pkl
	pr: 0.500 - l: 1
	Entrenando capa  1
	Entrenado en 0.82m

	Guardando modelo en models/msda/twitter_3.pkl
	pr: 0.500 - l: 3
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenado en 2.05m

	Guardando modelo en models/msda/twitter_4.pkl
	pr: 0.500 - l: 5
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenando capa  4
	Entrenando capa  5
	Entrenado en 3.28m

	Guardando modelo en models/msda/twitter_5.pkl
	pr: 0.800 - l: 1
	Entrenando capa  1
	Entrenado en 0.77m

	Guardando modelo en models/msda

```python
domain_msda_paths = [ruta_1,...,ruta_n]
```

In [7]:
best_msda_paths = os.path.join(models_path, tipo, "best_%s_paths.pkl" % dataset_name )
if os.path.exists(best_msda_paths):
    #se cargan las rutas de los modelos guardados
    domain_msda_paths = joblib.load(best_msda_paths)   
else:
    domain_msda_paths = []
    #se obtienen los mejores modelos 
    for domain in domains:
        print "Obteniendo mejor clasificador para %s" % domain
        i = 1

        best_score = 0
        best_model = None
        for msda_model_path in saved_paths:
            
            msda_model = joblib.load(msda_model_path)
            theano_model = msda_model['model']
            pr = msda_model['pr']
            n_layers = msda_model['l']

            print "%d) l=%d, pr=%.3f " % (i, msda_model['l'], msda_model['pr']),

            # se obtienen los datos del dominio
            X_tr = np.asarray(labeled[domain]['X_tr'].todense())
            y_tr = np.asarray(labeled[domain]['y_tr'].todense()).argmax(axis=1)

            # se adaptan los datos
            X_deep = theano_model.predict(X_tr)

            # se obtiene el mejor score con GridSearch
            new_clf = get_best_score(X_deep, y_tr, clasifier='SVC', n_jobs=4)
            new_score = new_clf.best_score_
            print "score: %.4f" % new_score
            # se guarda si es el mejor para el modelo
            if new_score > best_score:
                best_score = new_score
                best_model = msda_model

            i = i+1

        #se guarda el mejor modelo para este dominio
        print "Mejor modelo: l=%d, pr=%.3f" % (best_model['l'], best_model['pr'])
        best_msda_path = os.path.join(models_path, tipo, "%s_%s.pkl" % (dataset_name, domain))
        print "Guardando en %s" % best_msda_path
        joblib.dump(best_model, best_msda_path)
        
        domain_msda_paths.append(best_msda_path)
    
    print "Guardando rutas en %s" % best_msda_paths
    joblib.dump(domain_msda_paths, best_msda_paths)

Obteniendo mejor clasificador para rio2016
1) l=1, pr=0.300  score: 0.8211
2) l=3, pr=0.300 

/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with Standar

 score: 0.8000
3) l=5, pr=0.300 

/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with Standar

 score: 0.7632
4) l=1, pr=0.500  score: 0.8263
5) l=3, pr=0.500  score: 0.7605
6) l=5, pr=0.500  score: 0.7026
7) l=1, pr=0.800  score: 0.8132
8) l=3, pr=0.800  score: 0.6605
9) l=5, pr=0.800  score: 0.6237
Mejor modelo: l=1, pr=0.500
Guardando en models/msda/twitter_rio2016.pkl
Obteniendo mejor clasificador para thevoice
1) l=1, pr=0.300  score: 0.8147
2) l=3, pr=0.300 

/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with Standar

 score: 0.7857
3) l=5, pr=0.300 

/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with Standar

 score: 0.7143
4) l=1, pr=0.500  score: 0.8147
5) l=3, pr=0.500  score: 0.7664
6) l=5, pr=0.500  score: 0.7008
7) l=1, pr=0.800  score: 0.8012
8) l=3, pr=0.800  score: 0.7046
9) l=5, pr=0.800  score: 0.6506
Mejor modelo: l=1, pr=0.300
Guardando en models/msda/twitter_thevoice.pkl
Guardando rutas en models/msda/best_twitter_paths.pkl


In [8]:
domain_msda_paths

['models/msda/twitter_rio2016.pkl', 'models/msda/twitter_thevoice.pkl']

In [9]:
#diccionario para mantener los dominios adaptados
adapted = {}

i = 0
for domain in domains:
    best_msda_path = os.path.join(models_path, tipo, "%s_%s.pkl" % (dataset_name, domain))
    print best_msda_path
    if os.path.exists(best_msda_path):
        print "Cargando mejor modelo para %s" % domain
        msda_model = joblib.load(best_msda_path)
        #theano_model = msda_model['model']
        
        print "\tAdaptando dominio"
        X_tr = np.asarray(labeled[domain]['X_tr'].todense())
        X_ts = np.asarray(labeled[domain]['X_ts'].todense())
        
        
        tr_reps = theano_model.predict(X_tr)
        ts_reps = theano_model.predict(X_ts)

        adapted[domain] = {
            'X_tr': tr_reps,
            'X_ts': ts_reps
        }
        
        i = i+1
    else:
        print "\tGenerar mejor modelo para %s" % domain
        
print "%d/%d dominios adaptados" % (i, len(domains))

models/msda/twitter_rio2016.pkl
Cargando mejor modelo para rio2016
	Adaptando dominio
models/msda/twitter_thevoice.pkl
Cargando mejor modelo para thevoice
	Adaptando dominio
2/2 dominios adaptados


In [10]:
%%time

df = pd.DataFrame(columns=dataframe_columns)

i=0
# por cada par posible para adaptar
for src in domains:
    for tgt in domains:
        if src is not tgt:
            print "Tarea %d de 2" % (i+1)
            
            #baseline in-domain error
            #e_b(T,T)
            #entrenado en dominio tgt y probado en dominio tgt
            X_tr = np.asarray(labeled[tgt]['X_tr'].todense())
            y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se crean las rutas para cargar o crear los modelos
            #model_name = "%s_%s_%s.pkl" % (tipo,src, tgt)
            model_name = "indomain_%s.pkl" % (tgt)
            model_path = os.path.join(models_path, dataset_name, model_name)
            
            
            #Se realiza una clasificacion, estimando los parametros mediante cv
            #svc = get_best_score(X_tr, y_tr)
            svc = load_best_score(model_path, X_tr, y_tr)
            b_error = 1-svc.score(X_ts, y_ts)
            
            #transfer error
            #entrenado en dominio src y probado en dominio tgt adaptados
            X_tr_a = adapted[src]['X_tr']
            y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
            
            X_ts_a = adapted[tgt]['X_ts']
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            print "Entrenando clasificador adaptado."
            svc_a = get_best_score(X_tr_a, y_tr, clasifier='SVC', n_jobs=4)
            #svc_a = SVC().fit(X_tr_a, y_tr)
            
            t_error = 1-svc_a.score(X_ts_a, y_ts)
            
            # transfer loss t
            # t_error - b_error
            t_loss = t_error - b_error
            
            tarea = src[0]+'->'+tgt[0]
            df.loc[i] = ['mSDA',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]
            
            i+=1

print "\nPruebas completadas"

Tarea 1 de 2
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 2 de 2
Cargando modelo existente.
Entrenando clasificador adaptado.

Pruebas completadas
CPU times: user 1.7 s, sys: 100 ms, total: 1.8 s
Wall time: 5.35 s


In [11]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,mSDA,r->t,rio2016,thevoice,16.153846,47.692308,31.538462
1,mSDA,t->r,thevoice,rio2016,19.791667,43.750000,23.958333


In [12]:
new_scores_path = os.path.join(scores_path, tipo, dataset_name+".csv")
print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/msda/twitter.csv
Resultados guardados.
